# Week 12: ML Review

### Setup

Run the following 2 cells to import all necessary libraries and helpers for this homework

In [ ]:
!wget -q https://github.com/PSAM-5020-2025F-A/5020-utils/raw/main/src/data_utils.py
!wget -q https://github.com/PSAM-5020-2025F-A/5020-utils/raw/main/src/image_utils.py
!wget -qO- https://github.com/PSAM-5020-2025F-A/5020-utils/releases/latest/download/lfw.tar.gz | tar xz

In [1]:
from random import randint

from sklearn.metrics import classification_report, confusion_matrix

from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

from data_utils import LFWUtils
from data_utils import classification_error, display_confusion_matrix
from image_utils import make_image

## Face Unlock

Let's train a model to detect our face. We can think of this as a simpler version of one of the components inside something like the face ID software on our phones.

We'll skip the face detection part, which is when we find faces in an image, and assume we can get cropped and aligned faces out of images or video streams. We'll look at face detection later in the semester.

This is a slightly different kind of problem from the classification exercise we did in class, but the process is mostly the same.

We will use a dataset with other people's faces, but in the end we are only interested on how well our model detects our face.

### We Always Start with the Data

The dataset we're using is inside `./data/images/lfw/cropped`. It's a subset of the [Labeled Faces in the Wild](https://vis-www.cs.umass.edu/lfw/) dataset.

Take a look at the directory.

What's there?

How's the data organized and labeled?

### Loading the Data

Since we're not interested in generic classification, and measuring how we do on unlabeled data, this whole dataset is labeled, and we can read it into `train` and `test` subsets by calling the `train_test_split()` function of the `LFWUtils` class.

This function takes an optional parameter that specifies what portion of the data should be used for the `test` dataset. We can start with the default value of $0.5$.

In [ ]:
train, test = LFWUtils.train_test_split(dir="./data/image/lfw/cropped", test_pct=0.5)

### Looking at the Data

Ok. Data is loaded.

What's in the data? How is it actually organized?

Take a look at the objects that were returned in each of the $2$ variables.

How big are our datasets?

Take a look at the `LABELS` and `L2I` members of the `LFWUtils` class.

In [ ]:
# TODO: look at dataset objects (train and test variables). What's in them?
# TODO: how big are them? (how many records?)
# TODO: how many labels do they have?
# TODO: what are the labels

### Visualizing the Data

We can open some random images to make sure the content of our datasets make sense:

In [ ]:
train_size = len(train["labels"])
ridx = randint(0, train_size - 1)

label_id = train["labels"][ridx]

print("id:", label_id,
      "\nlabel:", LFWUtils.LABELS[label_id],
      "\nfrom:", train["files"][ridx])

display(make_image(train["pixels"][ridx], width=LFWUtils.IMAGE_SIZE[0]))

### Adding your images

Create a directory in the `dataset` directory for your images. Give it a one-word name, like your last name, your New School id or your initials. For example, mine is called `tgh` and is located at: `./data/images/lfw/cropped/tgh`.

Now, add between $20$ and $30$ images of your face to your directory. 

The images should be just like the ones that are already there for the other people:
- $130$ pixels wide by 
- $170$ pixels tall
- single-channel grayscale
- jpeg format
- named `label-number.jpg` (for example: `tgh-000.jpg`)

Feel free to do this manually using Photoshop or any other image editing software, but the easiest way is to use this interface that automatically crops faces out of pictures and creates images in the correct format:

[Face Align](https://huggingface.co/spaces/visualizedata/PSAM5020-FaceAlign-Gradio)

It will also align the faces and put the eyes in a consistent location. There's even an option to capture pictures from a live camera stream.

### Reload Dataset

Just run the `train_test_split()` again.

### PCA, Classification, etc etc etc

Now that we have added our images to the dataset, let's train a classifier and see how well it performs on not just classification, but on recognizing our face.

We can aim for an explained variance value of about $80\%$, and adjust that later if we find necessary.

Once we have the PCs for our training dataset in a `DataFrame` we can add a `label` column to it with the correct labels we have in `train["labels"]`.

We can also create a `DataFrame` for testing now by using the same `PCA` object to `transform()` the `test["pixels"]` data.

Since we won't train anything with the test dataset, it's ok to just keep the labels in `test["labels"]` as they are.

In [ ]:
# TODO: create PCA, fit and transform train data
# TODO: check PCA captured variance
# TODO: prepare DataFrame for training (add label column)
# TODO: create the test DataFrame by running PCA on the test data

We can use the following cell to take a look at our images and their reconstructions.

This assumes the `DataFrame` is called `train_df` and the `PCA` object is called `face_pca`. Adjust these if necessary.

In [ ]:
train_size = len(train["labels"])
ridx = randint(0, train_size - 1)

# reconstruct image
pca_pixels = face_pca.inverse_transform(train_df.iloc[[ridx]].drop(columns=["label"]))

display(make_image(train["pixels"][ridx], width=LFWUtils.IMAGE_SIZE[0]))
display(make_image(pca_pixels, width=LFWUtils.IMAGE_SIZE[0]))

In [ ]:
# filter the DataFrame by our label
awesome_df = train_df[train_df["label"] == LFWUtils.L2I["watts"]]

# save index of first image with our label
awesome_idx = awesome_df.index[0]

# reconstruct image
pca_pixels = face_pca.inverse_transform(awesome_df.iloc[[0]].drop(columns=["label"]))

display(make_image(train["pixels"][awesome_idx], width=LFWUtils.IMAGE_SIZE[0]))
display(make_image(pca_pixels, width=LFWUtils.IMAGE_SIZE[0]))

### Interpretation

<span style="color:hotpink;">
Do these make sense ? Do they look "recognizable" ? How do they change as a function of <code>n_components</code> ?
</span>

Now, back to classifying...

Maybe start with `RandomForestClassifier()` or `SVC()`...

In [ ]:
# TODO: create a classifier
# TODO: separate input and output columns from the train DataFrame
# TODO: train model using train data and labels
# TODO: run prediction on train data

### Validate model with training data

In [ ]:
# measure classification error
print("error:", classification_error(train["labels"], train_predictions))

# look at precision/recall from classification_report
print(classification_report(train["labels"], train_predictions))

# look at confusion matrix
display_confusion_matrix(train["labels"], train_predictions, LFWUtils.LABELS)

### Interpretation

<span style="color:hotpink;">
How does the confusion matrix look ? What does it mean ?
</span>

### Validate model with testing data

In [ ]:
# TODO: run prediction on test data
# TODO: measure classification error
# TODO: look at precision/recall from classification_report
# TODO: look at confusion matrix

### Different Classifiers

Try changing the classifier type above, or some of its parameters, to see if the overall accuracy can be improved.

Some things to try:
- `SVC(kernel="linear")`: this changes the type of curve the model tries to use to separate our classes. The more complex default type (`rbf`) might be over-fitting the data. We can experiment with `linear` or `poly` curves.
- `LogisticRegression()`: the classification that fits statistical modeling functions (bell curves) to our data.


### Hyper-parameter Optimization

Each model type has a handful of parameters that can be adjusted and optimized.

For example, we could add "regularization" to a `LogisticRegression` classifier with `C=0.1`.

Regularization is a process that makes training harder by setting more restrictions on the kind of answers it can give.

This can be restrictions like:
- keep all calculated parameters close to $0$
- minimize the number of non-zero parameters used
- keep all parameters positive

The result is slightly worse performance on the training dataset, but hopefully better generalization of the model and better performance on the test dataset.

In [ ]:
# create a classifier
model = LogisticRegression(solver="liblinear", C=0.1, random_state=1010)

# train model using train data and labels
model.fit(train_features, train_df["label"])

# run prediction
train_predictions = model.predict(train_features)
test_predictions = model.predict(test_df)

# measure classification error
print("error:", classification_error(train["labels"], train_predictions))
print("error:", classification_error(test["labels"], test_predictions))

# look at precision/recall and confusion matrix
print(classification_report(test["labels"], test_predictions))
display_confusion_matrix(test["labels"], test_predictions, LFWUtils.LABELS)

### Interpretation

<span style="color:hotpink;">
How does THIS confusion matrix look ? What does it mean ? How does it perform for your pictures ?
</span>

### Precision and Recall

Accuracy, which is the complement of our `classification_error` value, is the measurement that is optimized during the `RandomForestClassifier` training process.

If we were training a regular classifier, we would look at `accuracy` (or `classification_error`) to determine if our model's performance is acceptable.

Since we're working on a personal face recognition model, we don't really care about overall accuracy, but instead are more interested in the `precision` and `recall` values for the classification of our particular images.

We don't want overall accuracy to be horrible, but we can be more specific in this case and be happy if the correct portion of our confusion matrix looks good.

Calculate the `precision` and `recall` values for the classification of your images.

In [ ]:
# TODO: calculate precision
# TODO: calculate recall

### Interpretation

<span style="color:hotpink;">
How is it performing for your images ? Which value, precision or recall, is higher ? What does that mean ?
</span>

We can run the following cell to see which classes have the highest `precision` and `recall` scores:

In [ ]:
print("top precision:", LFWUtils.top_precision(test["labels"], test_predictions, top=5))
print("top recall:", LFWUtils.top_recall(test["labels"], test_predictions, top=5))